<a href="https://colab.research.google.com/github/Lakshmi-krishna-vr/TeamLuminous-ImageSharpeningUsingKnowledgeDistillation/blob/main/Team_Luminous_knowledge_Distillation_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download and unzip the DIV2K dataset
url = "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip"
print("📥 Downloading DIV2K HR images...")
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("DIV2K_train_HR")
print("✅ Dataset downloaded and extracted!")


In [ ]:
# --- Data Splitting ---
# Split the dataset for training and testing to ensure fair evaluation
DATASET_PATH = 'DIV2K_train_HR/DIV2K_train_HR'
all_files = sorted(glob.glob(os.path.join(DATASET_PATH, "*.png")))

# Use 200 images for training and 100 for testing, as requested
train_files = all_files[:700]
test_files = all_files[700:800] # A benchmark set of 100 images

print(f"Found {len(train_files)} training images.")
print(f"Found {len(test_files)} testing images.")


Found 700 training images.
Found 100 testing images.


In [ ]:
class SharpeningDataset(Dataset):
    """Custom Dataset for creating blurry/sharp image pairs."""
    def __init__(self, image_paths, patch_size=256):
        self.image_paths = image_paths
        self.patch_size = patch_size
        self.to_tensor = ToTensor()
        self.scale_factor = 2 # Defines the initial downscaling factor

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            # Load the high-resolution (HR) ground truth image
            hr_image = Image.open(self.image_paths[idx]).convert('RGB')
            w, h = hr_image.size

            # Ensure image is large enough, resize if necessary
            if w < self.patch_size or h < self.patch_size:
                hr_image = hr_image.resize((self.patch_size, self.patch_size), Image.BICUBIC)
                w, h = hr_image.size

            # Take a random crop for data augmentation
            rand_w = np.random.randint(0, w - self.patch_size + 1)
            rand_h = np.random.randint(0, h - self.patch_size + 1)
            hr_patch = hr_image.crop((rand_w, rand_h, rand_w + self.patch_size, rand_h + self.patch_size))

            # Create the blurry (LR) input image
            # 1. Downscale the image
            lr_size = (self.patch_size // self.scale_factor, self.patch_size // self.scale_factor)
            lr_patch = hr_patch.resize(lr_size, Image.BICUBIC)
            # 2. Apply a strong Gaussian blur
            lr_patch = lr_patch.filter(ImageFilter.GaussianBlur(radius=1.5))

            # 3. Upscale it back to the original patch size
            lr_patch = lr_patch.resize((self.patch_size, self.patch_size), Image.BICUBIC)

            # Convert images to PyTorch tensors
            hr_tensor = self.to_tensor(hr_patch)
            lr_tensor = self.to_tensor(lr_patch)
            return {'lr': lr_tensor, 'hr': hr_tensor}
        except Exception as e:
            print(f"[Dataset Error] at index {idx}: {e}")
            raise e  # Let it crash visibly if needed

# Create DataLoaders
patch_size = 256
batch_size = 5

train_dataset = SharpeningDataset(train_files, patch_size=patch_size)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

test_dataset = SharpeningDataset(test_files, patch_size=patch_size)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0) # Batch size 1 for testing


In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)


True
12.1


In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
!nvidia-smi



Thu Jul  3 17:30:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.99                 Driver Version: 555.99         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   43C    P3             11W /   40W |       8MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
class TeacherModel(nn.Module):
    """A high-capacity model to serve as the 'teacher'."""
    def __init__(self):
        super(TeacherModel, self).__init__()
        # Deeper and wider architecture for higher performance
        self.layers = nn.Sequential(
            nn.Conv2d(3, 384, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(192, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(192, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(96, 48, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(48, 3, kernel_size=3, padding=1)

        )

    def forward(self, x):
        # Residual connection helps stabilize training
        return x + self.layers(x)

class StudentModel(nn.Module):
    """An ultra-lightweight model to be trained via distillation."""
    def __init__(self):
        super(StudentModel, self).__init__()
        # Fewer layers and channels to be lightweight and fast
        self.layers = nn.Sequential(
            nn.Conv2d(3, 128, kernel_size=3, padding=1),     # ~3.6K
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 64, kernel_size=1),               # ~8.3K
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, padding=1)
        )

    def forward(self, x):
        # Residual connection is crucial for good performance
        return x + self.layers(x)

# Instantiate models to check parameter counts
teacher = TeacherModel()
student = StudentModel()

teacher_params = sum(p.numel() for p in teacher.parameters() if p.requires_grad)
student_params = sum(p.numel() for p in student.parameters() if p.requires_grad)

print(f"Teacher Model Parameters: {teacher_params:,}")
print(f"Student Model Parameters: {student_params:,}")
print(f"The student model is ~{teacher_params/student_params:.1f}x smaller than the teacher model.")



Teacher Model Parameters: 2,561,187
Student Model Parameters: 13,571
The student model is ~188.7x smaller than the teacher model.
